In [1]:
setwd("/data/Projects/ACC_NCDR/NCDR/BJMDATA/ACTION/")
load(file='/data/Projects/ACC_NCDR/NCDR/BJMDATA/ACTION/RData/patientmerged_meds.RData')

In [2]:
#Exclude patients transferred to another acute care facility
cohort.data <- cohort.data[-which(cohort.data$DCLocation == 3) ,]

In [3]:
#Dates for cohort selection, starts at 2011 to account for CAFirstMedCon coding
cohort.data$DATE <- as.Date(as.POSIXct(cohort.data$Timeframe, origin = "1960-01-01"))
start.date <- as.Date(as.POSIXct("2011-01-01"))
end.date <- as.Date(as.POSIXct("2017-01-01"))
timeinterval.data <- cohort.data[which(cohort.data$DATE > start.date & cohort.data$DATE <= end.date) ,]

In [2]:
race.categorical.variables <- c('RaceWhite',
                                'Sex',
                                'RaceBlack',
                                'RaceAsian',
                                'RaceAmIndian',
                                'RaceNatHaw',
                                'HispOrig',
                                'DATE')

lab.categorical.variables <- c('InitTrop',
                               'InitCKMB',
                               'InitCreat',
                               'InitHGB',
                               'Lipids',
                               'BNP',
                               'proBNP',
                               'PosMarkers')

cardiac.categorical.variables <- c('ECGFindings',
                                   'StemiNoted',
                                   'OtherECGFindings',
                                   'HFFirstMedCon',
                                   'ShockFirstMedCon',
                                   #'Cocaine',
                                   'CAFirstMedCon')

history.categorical.variables <- c('Hypertension',
                                   'Smoker',
                                   'Dyslipidemia',
                                   'CurrentDialysis',
                                   'ChronicLungDisease', #Note- comment out for Julian's model,  keep in for McNamara
                                   'Diabetes',
                                   'DiabetesControl',
                                   'PriorMI',
                                   'PriorHF',
                                   'PriorPCI',
                                   'PriorCABG',
                                   'PriorAfib',
                                   'PriorCVD',
                                   'PriorStroke',
                                   'PriorTIA',
                                   'PriorPAD'
                                   )

inhosp.categorical.variables <- c('CE_CardioShock',
                                  'CE_HF',
                                  'CE_CardiacArrest',
                                  'CE_Bleeding',
                                  'CE_RBC')

meds.home.categorical.variables <- names(cohort.data)[which(grepl("Home", names(cohort.data)))]

all.categorical.variables <- c(race.categorical.variables, 
                               lab.categorical.variables,
                               cardiac.categorical.variables,
                               history.categorical.variables, 
                              # inhosp.categorical.variables,
                               meds.home.categorical.variables
)

continuous.variables <- c('InitTropValue',
                              'InitTropURL',
                              #'InitCKMBValue',
                              #'InitCKMBULN',
                              'InitCreatValue',
                              'InitHGBValue',
                              #'InitINRValue',
                              #'LipidsTC',
                              #'LipidsHDL',
                              #'LipidsLDL',
                              #'LipidsTrig',
                          'Age',
                          'Weight',
                          'Height',
                          'HRFirstMedCon',
                          'SBPFirstMedCon'
)

In [4]:
sample.data <- timeinterval.data

In [5]:
exclude_data <- sample.data[which(is.na(sample.data$PriorPCI)) ,]

In [6]:
include_data <- sample.data[which(!is.na(sample.data$PriorPCI)) ,]

In [7]:
#data <- list(exclude_data, include_data)
data <- list(exclude_data)

In [8]:
summary(exclude_data)

 SubmissionPatientKey   EpisodeKey          ClientKey     CMSCMTiming    
 Min.   : 323896      Min.   : 10407338   Min.   :   2   Min.   :1.00    
 1st Qu.:1962910      1st Qu.: 40001001   1st Qu.: 613   1st Qu.:4.00    
 Median :3575350      Median :102619327   Median :1430   Median :4.00    
 Mean   :3677134      Mean   :104065071   Mean   :1353   Mean   :3.87    
 3rd Qu.:5335156      3rd Qu.:165439960   3rd Qu.:2120   3rd Qu.:4.00    
 Max.   :6991844      Max.   :220721989   Max.   :4142   Max.   :4.00    
                                                         NA's   :145948  
  DC_CardRehab      DC_Diet        DC_Exercise      DCLocation   
 Min.   :0.000   Min.   :0.00     Min.   :0.00    Min.   :1.000  
 1st Qu.:1.000   1st Qu.:1.00     1st Qu.:1.00    1st Qu.:1.000  
 Median :1.000   Median :1.00     Median :1.00    Median :1.000  
 Mean   :1.025   Mean   :0.94     Mean   :0.85    Mean   :1.283  
 3rd Qu.:1.000   3rd Qu.:1.00     3rd Qu.:1.00    3rd Qu.:1.000  
 Max.   :7.0

In [9]:
summary(include_data)

 SubmissionPatientKey   EpisodeKey          ClientKey     CMSCMTiming    
 Min.   : 308635      Min.   :  9660279   Min.   :  14   Min.   :1.0     
 1st Qu.:1916745      1st Qu.: 36009652   1st Qu.: 711   1st Qu.:4.0     
 Median :3573720      Median :102616872   Median :1256   Median :4.0     
 Mean   :3657823      Mean   :103604209   Mean   :1333   Mean   :3.8     
 3rd Qu.:5278084      3rd Qu.:150882382   3rd Qu.:1979   3rd Qu.:4.0     
 Max.   :6991856      Max.   :220723204   Max.   :5102   Max.   :4.0     
                                                         NA's   :597646  
  DC_CardRehab      DC_Diet        DC_Exercise       DCLocation   
 Min.   :0.00    Min.   :0        Min.   :0.0      Min.   :1.00   
 1st Qu.:1.00    1st Qu.:1        1st Qu.:1.0      1st Qu.:1.00   
 Median :1.00    Median :1        Median :1.0      Median :1.00   
 Mean   :1.08    Mean   :1        Mean   :0.9      Mean   :1.25   
 3rd Qu.:1.00    3rd Qu.:1        3rd Qu.:1.0      3rd Qu.:1.00   
 Max. 

In [53]:
for (subset.data in data){
    subset.data$DCStatus[which(subset.data$DCStatus == 1)] <- 0
    subset.data$DCStatus[which(subset.data$DCStatus == 2)] <- 1
    
    subset.data$InitTrop[which(subset.data$InitTrop == 2)] <- 1
    
    subset.data$PriorStroke[which(subset.data$PriorCVD == 0)] <- 0
    subset.data$PriorTIA[which(subset.data$PriorCVD == 0)] <- 0
    
    #Divide STEMI variable into separate subclasses based on ECGFindings variable
    subset.data$Stemi_ST <- 0
    subset.data$Stemi_LBBB <- 0
    subset.data$Stemi_PostMI <- 0
    
    subset.data$Stemi_ST[which(subset.data$ECGFindings == 1)] <- 1
    subset.data$Stemi_LBBB[which(subset.data$ECGFindings == 2)] <- 1
    subset.data$Stemi_PostMI[which(subset.data$ECGFindings == 3)] <- 1
    
    #Divide non-stemi variable into separate subclasses based on OtherECGFindings variable
    subset.data$NoStemi_STdep <- 0
    subset.data$NoStemi_Twave <- 0
    subset.data$NoStemi_transST <- 0
    subset.data$NoStemi_none <- 0
    subset.data$NoStemi_oldLBBB <- 0
    subset.data$NoStemi_other <- 0 
    
    subset.data$NoStemi_STdep[which(subset.data$OtherECGFindings == 1)] <- 1
    subset.data$NoStemi_Twave[which(subset.data$OtherECGFindings == 2)] <- 1
    subset.data$NoStemi_transST[which(subset.data$OtherECGFindings == 3)] <- 1
    subset.data$NoStemi_none[which(subset.data$OtherECGFindings == 4)] <- 1
    #subset.data$NoStemi_none[which(is.na(subset.data$OtherECGFindings))] <- 1
    subset.data$NoStemi_oldLBBB[which(subset.data$OtherECGFindings == 5)] <- 1
    subset.data$NoStemi_other[which(subset.data$OtherECGFindings == 6)] <- 1
    
    #Divide diabetes into separate treatment groups diabetes control
    subset.data$DM_notx <- 0
    subset.data$DM_diet <- 0
    subset.data$DM_oral <- 0
    subset.data$DM_insulin <- 0 
    subset.data$DM_other <- 0
    
    subset.data$DM_notx[which(subset.data$DiabetesControl == 1)] <- 1
    subset.data$DM_diet[which(subset.data$DiabetesControl == 2)] <- 1
    subset.data$DM_oral[which(subset.data$DiabetesControl == 3)] <- 1
    subset.data$DM_insulin[which(subset.data$DiabetesControl == 4)] <- 1
    subset.data$DM_other[which(subset.data$DiabetesControl == 5)] <- 1
        
    drops <- c(
        "Dabigatran_Home",
        "Rivaroxaban_Home",
        "Apixaban_Home",
        "Tic_Home",
        "DiabetesControl",
        "DATE",
        "PriorTIA",
        "ASA_Home",
        "CLPD_Home",
        "Ticlid_Home",
        "ARB_Home",
        "BB_Home",
        "Statin_Home",
        "OLLA_Home",
        "Prasu_Home",
        "War_Home",
        "ABA_Home"
              )
    subset.data <- subset.data[, !(names(subset.data) %in% drops)]
    all.categorical.variables <- all.categorical.variables[! all.categorical.variables %in% drops]
        
    #BMI, max = 60 kg/m^2
    subset.data$BMI <- subset.data$Weight / ((subset.data$Height/100)^2)
    subset.data$BMI[which(subset.data$BMI < 15)] <- 15
    subset.data$BMI[which(subset.data$BMI > 60)] <- 60 
    
    #CrCl
    subset.data$CrCl <- 0
    subset.data$CrCl[which(subset.data$Sex == 2)] <- (   (140-subset.data$Age[which(subset.data$Sex == 2)]) * subset.data$Weight[which(subset.data$Sex == 2)]  ) / (subset.data$InitCreatValue[which(subset.data$Sex == 2)] * 72) * 0.85  
    subset.data$CrCl[which(subset.data$Sex == 1)] <- (   (140-subset.data$Age[which(subset.data$Sex == 1)]) * subset.data$Weight[which(subset.data$Sex == 1)]  ) / (subset.data$InitCreatValue[which(subset.data$Sex == 1)] * 72)
    subset.data$CrCl[which(subset.data$CrCl >= 200)] <- 200
    
    #calculate troponin ratio, max 20
    subset.data$TropRatio <- subset.data$InitTropValue / subset.data$InitTropURL
    subset.data$TropRatio[which(subset.data$TropRatio >= 20)] <- 20 
    #subset.data$TropRatio[which(is.na(subset.data$TropRatio))] <- 0

}

In [16]:
dim(include_data)

[1] 755402     63

In [17]:
dim(exclude_data)

[1] 191195     63

In [14]:
summary(exclude_data)

   RaceWhite           Sex          RaceBlack       RaceAsian      
 Min.   :0.0000   Min.   :1.000   Min.   :0.000   Min.   :0.00000  
 1st Qu.:1.0000   1st Qu.:1.000   1st Qu.:0.000   1st Qu.:0.00000  
 Median :1.0000   Median :1.000   Median :0.000   Median :0.00000  
 Mean   :0.8443   Mean   :1.347   Mean   :0.113   Mean   :0.02748  
 3rd Qu.:1.0000   3rd Qu.:2.000   3rd Qu.:0.000   3rd Qu.:0.00000  
 Max.   :1.0000   Max.   :2.000   Max.   :1.000   Max.   :1.00000  
                                                                   
  RaceAmIndian        RaceNatHaw          HispOrig           DATE           
 Min.   :0.000000   Min.   :0.000000   Min.   :0.0000   Min.   :2011-04-01  
 1st Qu.:0.000000   1st Qu.:0.000000   1st Qu.:0.0000   1st Qu.:2013-01-01  
 Median :0.000000   Median :0.000000   Median :0.0000   Median :2014-04-01  
 Mean   :0.006208   Mean   :0.002678   Mean   :0.0785   Mean   :2014-04-06  
 3rd Qu.:0.000000   3rd Qu.:0.000000   3rd Qu.:0.0000   3rd Qu.:2015-10

In [19]:
sd(exclude_data$InitCreatValue, na.rm =TRUE)

[1] 1.214771

In [21]:
sd(exclude_data$InitHGBValue, na.rm=TRUE)

[1] 2.201222

In [22]:
sd(exclude_data$Age, na.rm=TRUE)

[1] 13.73375

In [23]:
sd(exclude_data$Weight, na.rm=TRUE)

[1] 22.28206

In [24]:
sd(exclude_data$HRFirstMedCon, na.rm=TRUE)

[1] 24.2815

In [25]:
sd(exclude_data$SBPFirstMedCon, na.rm=TRUE)

[1] 35.79447

In [40]:
dim(subset.data)

[1] 755402     63

In [39]:
summary(subset.data)

   RaceWhite           Sex          RaceBlack        RaceAsian      
 Min.   :0.0000   Min.   :1.000   Min.   :0.0000   Min.   :0.00000  
 1st Qu.:1.0000   1st Qu.:1.000   1st Qu.:0.0000   1st Qu.:0.00000  
 Median :1.0000   Median :1.000   Median :0.0000   Median :0.00000  
 Mean   :0.8485   Mean   :1.344   Mean   :0.1153   Mean   :0.01926  
 3rd Qu.:1.0000   3rd Qu.:2.000   3rd Qu.:0.0000   3rd Qu.:0.00000  
 Max.   :1.0000   Max.   :2.000   Max.   :1.0000   Max.   :1.00000  
                                                                    
  RaceAmIndian        RaceNatHaw          HispOrig       InitTrop     
 Min.   :0.000000   Min.   :0.000000   Min.   :0.00   Min.   :0.0000  
 1st Qu.:0.000000   1st Qu.:0.000000   1st Qu.:0.00   1st Qu.:1.0000  
 Median :0.000000   Median :0.000000   Median :0.00   Median :1.0000  
 Mean   :0.007382   Mean   :0.001701   Mean   :0.06   Mean   :0.9888  
 3rd Qu.:0.000000   3rd Qu.:0.000000   3rd Qu.:0.00   3rd Qu.:1.0000  
 Max.   :1.000000   Ma

In [41]:
sd(subset.data$TropRatio, na.rm=TRUE)

[1] 8.11786

In [42]:
sd(subset.data$CrCl, na.rm=TRUE)

[1] 42.49563

In [43]:
sd(subset.data$InitCreatValue, na.rm=TRUE)

[1] 1.215301

In [44]:
sd(subset.data$InitHGBValue, na.rm=TRUE)

[1] 2.188574

In [45]:
sd(subset.data$BMI, na.rm=TRUE)

[1] 6.680252

In [46]:
sd(subset.data$HRFirstMedCon, na.rm=TRUE)

[1] 23.94665

In [47]:
sd(subset.data$SBPFirstMedCon, na.rm=TRUE)

[1] 35.16113

In [48]:
sd(subset.data$Age, na.rm=TRUE)

[1] 13.65551

In [49]:
sd(subset.data$Weight, na.rm=TRUE)

[1] 22.05579

In [42]:
sd(subset.data$CrCl, na.rm=TRUE)

[1] 42.49563

In [42]:
sd(subset.data$CrCl, na.rm=TRUE)

[1] 42.49563

In [15]:
summary(include_data)

   RaceWhite           Sex          RaceBlack        RaceAsian      
 Min.   :0.0000   Min.   :1.000   Min.   :0.0000   Min.   :0.00000  
 1st Qu.:1.0000   1st Qu.:1.000   1st Qu.:0.0000   1st Qu.:0.00000  
 Median :1.0000   Median :1.000   Median :0.0000   Median :0.00000  
 Mean   :0.8485   Mean   :1.344   Mean   :0.1153   Mean   :0.01926  
 3rd Qu.:1.0000   3rd Qu.:2.000   3rd Qu.:0.0000   3rd Qu.:0.00000  
 Max.   :1.0000   Max.   :2.000   Max.   :1.0000   Max.   :1.00000  
                                                                    
  RaceAmIndian        RaceNatHaw          HispOrig         DATE           
 Min.   :0.000000   Min.   :0.000000   Min.   :0.00   Min.   :2011-04-01  
 1st Qu.:0.000000   1st Qu.:0.000000   1st Qu.:0.00   1st Qu.:2013-01-01  
 Median :0.000000   Median :0.000000   Median :0.00   Median :2014-04-01  
 Mean   :0.007382   Mean   :0.001701   Mean   :0.06   Mean   :2014-03-25  
 3rd Qu.:0.000000   3rd Qu.:0.000000   3rd Qu.:0.00   3rd Qu.:2015-10-01 

In [10]:
summary(subset.data)

   RaceWhite           Sex          RaceBlack        RaceAsian      
 Min.   :0.0000   Min.   :1.000   Min.   :0.0000   Min.   :0.00000  
 1st Qu.:1.0000   1st Qu.:1.000   1st Qu.:0.0000   1st Qu.:0.00000  
 Median :1.0000   Median :1.000   Median :0.0000   Median :0.00000  
 Mean   :0.8485   Mean   :1.344   Mean   :0.1153   Mean   :0.01926  
 3rd Qu.:1.0000   3rd Qu.:2.000   3rd Qu.:0.0000   3rd Qu.:0.00000  
 Max.   :1.0000   Max.   :2.000   Max.   :1.0000   Max.   :1.00000  
                                                                    
  RaceAmIndian        RaceNatHaw          HispOrig         DATE           
 Min.   :0.000000   Min.   :0.000000   Min.   :0.00   Min.   :2011-04-01  
 1st Qu.:0.000000   1st Qu.:0.000000   1st Qu.:0.00   1st Qu.:2013-01-01  
 Median :0.000000   Median :0.000000   Median :0.00   Median :2014-04-01  
 Mean   :0.007382   Mean   :0.001701   Mean   :0.06   Mean   :2014-03-25  
 3rd Qu.:0.000000   3rd Qu.:0.000000   3rd Qu.:0.00   3rd Qu.:2015-10-01 